# 🚗 Car Price Prediction with Machine Learning
## CodeAlpha Data Science Internship — Task 3

| Detail | Info |
|--------|------|
| **Intern** | Mohammed Abuzar |
| **Company** | CodeAlpha |
| **Task** | Task 3 — Car Price Prediction |
| **Best Model** | Gradient Boosting (R²=0.9699) |

In [ ]:
import os
if not os.path.exists('car_data.csv'):
    from google.colab import files
    print('Upload car_data.csv...')
    uploaded = files.upload()
    print('Uploaded:', list(uploaded.keys()))
else:
    print('Dataset found!')

---
## 📦 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
plt.style.use('seaborn-v0_8-whitegrid')
COLORS = ['#3498db','#e74c3c','#2ecc71','#f39c12']
print('All libraries imported!')

---
## 📂 2. Load & Explore Dataset

In [ ]:
df = pd.read_csv('car_data.csv')
print('Shape:', df.shape)
print('Columns:', df.columns.tolist())
print('Null values:', df.isnull().sum().sum())
df.head(5)

In [ ]:
print('Statistical Summary')
print(df.describe().round(2))
print()
for col in df.select_dtypes(include='object').columns:
    print(f'{col}: {df[col].unique().tolist()}')

---
## 🔍 3. Exploratory Data Analysis (EDA)

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(15,5))
fig.suptitle('Car Selling Price Distribution Analysis', fontsize=15, fontweight='bold')
axes[0].hist(df['Selling_Price'], bins=30, color='#3498db', edgecolor='white', alpha=0.85)
axes[0].axvline(df['Selling_Price'].mean(), color='#e74c3c', linestyle='--', linewidth=2, label=f"Mean: {df['Selling_Price'].mean():.1f}L")
axes[0].axvline(df['Selling_Price'].median(), color='#2ecc71', linestyle='--', linewidth=2, label=f"Median: {df['Selling_Price'].median():.1f}L")
axes[0].set_title('Overall Distribution', fontsize=12, fontweight='bold'); axes[0].legend()
fuel_avg = df.groupby('Fuel_Type')['Selling_Price'].mean().sort_values(ascending=False)
bars = axes[1].bar(fuel_avg.index, fuel_avg.values, color=['#e74c3c','#3498db','#2ecc71'], alpha=0.85, edgecolor='white')
for b,v in zip(bars,fuel_avg.values): axes[1].text(b.get_x()+b.get_width()/2., b.get_height()+0.1, f'{v:.1f}L', ha='center', fontsize=11, fontweight='bold')
axes[1].set_title('Avg Price by Fuel Type', fontsize=12, fontweight='bold')
trans_avg = df.groupby('Transmission')['Selling_Price'].mean()
bars2 = axes[2].bar(trans_avg.index, trans_avg.values, color=['#9b59b6','#f39c12'], alpha=0.85, edgecolor='white', width=0.5)
for b,v in zip(bars2,trans_avg.values): axes[2].text(b.get_x()+b.get_width()/2., b.get_height()+0.1, f'{v:.1f}L', ha='center', fontsize=12, fontweight='bold')
axes[2].set_title('Avg Price by Transmission', fontsize=12, fontweight='bold')
plt.tight_layout(); plt.savefig('price_distribution.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: price_distribution.png')

In [ ]:
df_tmp = df.copy(); df_tmp['Car_Age'] = 2024 - df_tmp['Year']
fig, axes = plt.subplots(2,2,figsize=(14,10))
fig.suptitle('EDA Key Insights', fontsize=15, fontweight='bold')
axes[0,0].scatter(df['Present_Price'], df['Selling_Price'], alpha=0.6, color='#3498db', edgecolors='white', s=60)
m,b = np.polyfit(df['Present_Price'], df['Selling_Price'], 1)
xl = np.linspace(df['Present_Price'].min(), df['Present_Price'].max(), 100)
axes[0,0].plot(xl, m*xl+b, color='#e74c3c', linewidth=2)
axes[0,0].set_title('Selling vs Present Price', fontsize=12, fontweight='bold'); axes[0,0].set_xlabel('Present Price'); axes[0,0].set_ylabel('Selling Price')
axes[0,1].scatter(df_tmp['Car_Age'], df['Selling_Price'], alpha=0.6, color='#e74c3c', edgecolors='white', s=60)
axes[0,1].set_title('Selling Price vs Car Age', fontsize=12, fontweight='bold'); axes[0,1].set_xlabel('Car Age (Years)'); axes[0,1].set_ylabel('Selling Price')
fuel_order = df.groupby('Fuel_Type')['Selling_Price'].median().sort_values(ascending=False).index
fd = [df[df['Fuel_Type']==f]['Selling_Price'].values for f in fuel_order]
bp = axes[1,0].boxplot(fd, labels=fuel_order, patch_artist=True, medianprops={'color':'white','linewidth':2})
for patch, c in zip(bp['boxes'], ['#e74c3c','#3498db','#2ecc71']): patch.set_facecolor(c); patch.set_alpha(0.7)
axes[1,0].set_title('Price by Fuel Type', fontsize=12, fontweight='bold'); axes[1,0].set_ylabel('Selling Price')
pd_pct = (df['Present_Price']-df['Selling_Price'])/df['Present_Price']*100
axes[1,1].hist(pd_pct, bins=25, color='#9b59b6', edgecolor='white', alpha=0.85)
axes[1,1].axvline(pd_pct.mean(), color='#e74c3c', linestyle='--', linewidth=2, label=f'Avg Drop: {pd_pct.mean():.1f}%')
axes[1,1].set_title('Price Depreciation %', fontsize=12, fontweight='bold'); axes[1,1].legend()
plt.tight_layout(); plt.savefig('eda_insights.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: eda_insights.png')

In [ ]:
num_cols = ['Selling_Price','Present_Price','Driven_kms','Year','Owner']
fig, ax = plt.subplots(figsize=(9,7))
mask = np.triu(np.ones_like(df[num_cols].corr(), dtype=bool))
sns.heatmap(df[num_cols].corr(), mask=mask, annot=True, fmt='.2f', cmap='RdYlGn', center=0, square=True, linewidths=0.5, ax=ax, annot_kws={'size':11,'weight':'bold'})
ax.set_title('Feature Correlation Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout(); plt.savefig('correlation_heatmap.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: correlation_heatmap.png')

---
## ⚙️ 4. Feature Engineering & Preprocessing

In [ ]:
df['Car_Age'] = 2024 - df['Year']
df['Price_Drop'] = df['Present_Price'] - df['Selling_Price']
df['Price_Drop_Pct'] = (df['Price_Drop'] / df['Present_Price']) * 100
df['KMs_per_Year'] = df['Driven_kms'] / df['Car_Age'].replace(0,1)
print('New features added:')
print(df[['Car_Age','Price_Drop','Price_Drop_Pct','KMs_per_Year']].describe().round(2))

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(16,5))
fig.suptitle('Feature Engineering Analysis', fontsize=15, fontweight='bold')
brand_avg = df.groupby('Car_Name')['Selling_Price'].mean().sort_values(ascending=False).head(12)
axes[0].barh(brand_avg.index[::-1], brand_avg.values[::-1], color='#3498db', alpha=0.85, edgecolor='white')
axes[0].set_title('Top 12 Cars by Avg Price', fontsize=12, fontweight='bold'); axes[0].set_xlabel('Avg Selling Price (Lakhs)')
year_avg = df.groupby('Year')['Selling_Price'].mean().reset_index()
axes[1].plot(year_avg['Year'], year_avg['Selling_Price'], color='#2ecc71', linewidth=2.5, marker='o', markersize=8, markerfacecolor='white', markeredgewidth=2)
axes[1].fill_between(year_avg['Year'], year_avg['Selling_Price'], alpha=0.15, color='#2ecc71')
axes[1].set_title('Avg Price by Year', fontsize=12, fontweight='bold'); axes[1].set_xlabel('Manufacturing Year'); axes[1].set_ylabel('Avg Price')
oc = df['Owner'].value_counts().sort_index()
ol = [f'{int(o)} Owner{"s" if o>1 else ""}' for o in oc.index]
axes[2].bar(ol, oc.values, color=['#2ecc71','#f39c12','#e74c3c','#9b59b6'][:len(oc)], alpha=0.85, edgecolor='white')
for i,v in enumerate(oc.values): axes[2].text(i, v+1, str(v), ha='center', fontsize=12, fontweight='bold')
axes[2].set_title('Cars by Number of Owners', fontsize=12, fontweight='bold'); axes[2].set_ylabel('Count')
plt.tight_layout(); plt.savefig('feature_analysis.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: feature_analysis.png')

In [ ]:
df_model = df.copy()
le = LabelEncoder()
df_model['Fuel_Type']    = le.fit_transform(df['Fuel_Type'])
df_model['Selling_type'] = le.fit_transform(df['Selling_type'])
df_model['Transmission'] = le.fit_transform(df['Transmission'])
X = df_model[['Car_Age','Present_Price','Driven_kms','Fuel_Type','Selling_type','Transmission','Owner','KMs_per_Year']]
y = df_model['Selling_Price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)
print(f'Train: {X_train.shape} | Test: {X_test.shape}')

---
## 🤖 5. Model Training — 4 Regressors

In [ ]:
models = {
    'Linear Regression'  : LinearRegression(),
    'Decision Tree'      : DecisionTreeRegressor(max_depth=6, random_state=42),
    'Random Forest'      : RandomForestRegressor(n_estimators=100, max_depth=8, random_state=42),
    'Gradient Boosting'  : GradientBoostingRegressor(n_estimators=100, max_depth=4, random_state=42)
}
results = {}
print(f'{"Model":<25} {"R²":>8} {"MAE":>8} {"RMSE":>8}')
print('-'*55)
for name, model in models.items():
    model.fit(X_train_sc, y_train)
    y_pred = model.predict(X_test_sc)
    r2   = r2_score(y_test, y_pred)
    mae  = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    cv   = cross_val_score(model, X_train_sc, y_train, cv=5, scoring='r2')
    results[name] = {'model':model,'y_pred':y_pred,'r2':r2,'mae':mae,'rmse':rmse,'cv_mean':cv.mean(),'cv_std':cv.std()}
    print(f'{name:<25} {r2:>8.4f} {mae:>8.3f} {rmse:>8.3f}')

---
## 📊 6. Model Comparison & Evaluation

In [ ]:
model_names = list(results.keys())
r2s  = [results[m]['r2']  for m in model_names]
maes = [results[m]['mae'] for m in model_names]
cvs  = [results[m]['cv_mean'] for m in model_names]
fig, axes = plt.subplots(1,3,figsize=(16,6))
fig.suptitle('Model Performance Comparison', fontsize=15, fontweight='bold')
bars = axes[0].bar(model_names, r2s, color=COLORS, alpha=0.85, edgecolor='white')
for b,v in zip(bars,r2s): axes[0].text(b.get_x()+b.get_width()/2., b.get_height()+0.005, f'{v:.4f}', ha='center', fontsize=10, fontweight='bold')
axes[0].set_title('R² Score (Higher=Better)', fontsize=12, fontweight='bold'); axes[0].set_ylim(0,1.1)
axes[0].set_xticklabels([m.replace(' ','\n') for m in model_names], fontsize=9)
bars2 = axes[1].bar(model_names, maes, color=COLORS, alpha=0.85, edgecolor='white')
for b,v in zip(bars2,maes): axes[1].text(b.get_x()+b.get_width()/2., b.get_height()+0.02, f'{v:.3f}', ha='center', fontsize=10, fontweight='bold')
axes[1].set_title('MAE (Lower=Better)', fontsize=12, fontweight='bold'); axes[1].set_xticklabels([m.replace(' ','\n') for m in model_names], fontsize=9)
x=np.arange(len(model_names)); w=0.35
axes[2].bar(x-w/2, r2s, w, label='Test R²', color=COLORS, alpha=0.85, edgecolor='white')
axes[2].bar(x+w/2, cvs,  w, label='CV Mean R²', color=COLORS, alpha=0.45, edgecolor='white', hatch='//')
axes[2].set_xticks(x); axes[2].set_xticklabels([m.replace(' ','\n') for m in model_names], fontsize=9)
axes[2].set_title('Test vs CV R²', fontsize=12, fontweight='bold'); axes[2].legend()
plt.tight_layout(); plt.savefig('model_comparison.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: model_comparison.png')

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(14,12))
fig.suptitle('Actual vs Predicted Car Prices — All Models', fontsize=15, fontweight='bold')
for ax, (name, res), color in zip(axes.flatten(), results.items(), COLORS):
    yp = res['y_pred']
    ax.scatter(y_test, yp, alpha=0.65, color=color, edgecolors='white', linewidth=0.5, s=60)
    lims=[min(y_test.min(),yp.min())-1, max(y_test.max(),yp.max())+1]
    ax.plot(lims,lims,'k--',linewidth=2,alpha=0.7,label='Perfect Prediction')
    ax.set_xlabel('Actual Price (Lakhs)'); ax.set_ylabel('Predicted Price (Lakhs)')
    ax.set_title(f'{name}\nR²={res["r2"]:.4f} | MAE={res["mae"]:.3f}L', fontsize=12, fontweight='bold'); ax.legend()
plt.tight_layout(); plt.savefig('actual_vs_predicted.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: actual_vs_predicted.png')

In [ ]:
rf = results['Random Forest']['model']
feat_names = ['Car Age','Present Price','Driven KMs','Fuel Type','Selling Type','Transmission','Owner','KMs/Year']
imps = rf.feature_importances_
si = np.argsort(imps); sf=[feat_names[i] for i in si]; sv=imps[si]
fig, ax = plt.subplots(figsize=(10,6))
bars = ax.barh(sf, sv*100, color=['#e74c3c' if v>imps.mean()*100 else '#3498db' for v in sv*100], alpha=0.85, edgecolor='white')
for b,v in zip(bars,sv): ax.text(b.get_width()+0.3, b.get_y()+b.get_height()/2, f'{v*100:.1f}%', va='center', fontsize=11, fontweight='bold')
ax.set_title('Random Forest — Feature Importance', fontsize=14, fontweight='bold'); ax.set_xlabel('Importance (%)')
plt.tight_layout(); plt.savefig('feature_importance.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: feature_importance.png')

In [ ]:
best_name = max(results, key=lambda m: results[m]['r2'])
best_res  = results[best_name]
residuals = y_test - best_res['y_pred']
fig, axes = plt.subplots(1,3,figsize=(16,5))
fig.suptitle(f'Residuals Analysis — {best_name}', fontsize=14, fontweight='bold')
axes[0].scatter(best_res['y_pred'], residuals, alpha=0.65, color='#3498db', edgecolors='white', s=60)
axes[0].axhline(0, color='#e74c3c', linestyle='--', linewidth=2)
axes[0].set_title('Residuals vs Predicted', fontsize=12, fontweight='bold')
axes[1].hist(residuals, bins=25, color='#2ecc71', edgecolor='white', alpha=0.85)
axes[1].axvline(0, color='#e74c3c', linestyle='--', linewidth=2)
axes[1].set_title('Residuals Distribution', fontsize=12, fontweight='bold')
rmse_v = {m: results[m]['rmse'] for m in model_names}
axes[2].bar(rmse_v.keys(), rmse_v.values(), color=COLORS, alpha=0.85, edgecolor='white')
for i,(n,v) in enumerate(rmse_v.items()): axes[2].text(i, v+0.02, f'{v:.3f}', ha='center', fontsize=10, fontweight='bold')
axes[2].set_title('RMSE Comparison', fontsize=12, fontweight='bold'); axes[2].set_xticklabels([m.replace(' ','\n') for m in model_names], fontsize=9)
plt.tight_layout(); plt.savefig('residuals_analysis.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: residuals_analysis.png')

In [ ]:
best_model = results[best_name]['model']
ages = np.arange(1,16)
sim = pd.DataFrame({'Car_Age':ages,'Present_Price':10.0,'Driven_kms':15000*ages,'Fuel_Type':0,'Selling_type':0,'Transmission':0,'Owner':0,'KMs_per_Year':15000})
pred_sim = best_model.predict(scaler.transform(sim))
car_ages=[1,3,5,7,10]
fig, axes = plt.subplots(1,2,figsize=(14,6))
fig.suptitle('Price Prediction Insights', fontsize=15, fontweight='bold')
axes[0].plot(ages, pred_sim, color='#9b59b6', linewidth=2.5, marker='o', markersize=8, markerfacecolor='white', markeredgewidth=2)
axes[0].fill_between(ages, pred_sim, alpha=0.15, color='#9b59b6')
axes[0].set_title(f'Depreciation Over Time ({best_name})', fontsize=12, fontweight='bold'); axes[0].set_xlabel('Car Age (Years)'); axes[0].set_ylabel('Predicted Price (Lakhs)')
x=np.arange(len(car_ages)); w=0.2
for i,(mname,mres) in enumerate(results.items()):
    sd = pd.DataFrame({'Car_Age':car_ages,'Present_Price':10.0,'Driven_kms':[15000*a for a in car_ages],'Fuel_Type':0,'Selling_type':0,'Transmission':0,'Owner':0,'KMs_per_Year':15000})
    preds = mres['model'].predict(scaler.transform(sd))
    axes[1].bar(x+i*w, preds, w, label=mname.replace(' ','\n'), color=COLORS[i], alpha=0.85, edgecolor='white')
axes[1].set_xticks(x+w*1.5); axes[1].set_xticklabels([f'{a}yr' for a in car_ages])
axes[1].set_title('All Models: Price by Car Age', fontsize=12, fontweight='bold'); axes[1].legend(fontsize=8)
plt.tight_layout(); plt.savefig('price_prediction.png', dpi=100, bbox_inches='tight'); plt.show()
print('Saved: price_prediction.png')

---
## ✅ 7. Conclusions & Insights

### 🏆 Model Results
| Model | R² | MAE | RMSE |
|-------|-----|-----|------|
| Linear Regression | 0.8470 | 1.222L | 1.878L |
| Decision Tree | 0.9358 | 0.764L | 1.216L |
| Random Forest | 0.9599 | 0.626L | 0.962L |
| **Gradient Boosting** | **0.9699** | **0.519L** | **0.833L** |

### 🔑 Key Insights
1. **Present Price** is the strongest predictor of resale value
2. **Car Age** is the most impactful feature for depreciation
3. **Diesel cars** fetch higher resale prices than Petrol/CNG
4. **Automatic transmission** commands premium resale value
5. **Gradient Boosting** outperforms all other models with R²=0.9699
6. **Average depreciation is ~50%** over a car's lifetime

---
*🚗 CodeAlpha Data Science Internship | Task 3 — Car Price Prediction*

In [ ]:
print('='*55)
print('  CAR PRICE PREDICTION — FINAL SUMMARY')
print('='*55)
print(f'  Dataset       : {df.shape[0]} cars, {df.shape[1]} features')
print(f'  Train/Test    : {X_train.shape[0]} / {X_test.shape[0]}')
print()
for name,res in results.items():
    print(f'  {name:<25} R²={res["r2"]:.4f}  MAE={res["mae"]:.3f}L')
print()
print(f'  Best Model    : {best_name} (R²={results[best_name]["r2"]:.4f})')
print('='*55)
print('Task 3 Complete — CodeAlpha Internship!')